## Advanced data analysis using field data

There is data for __two pumps (Pump-ID1, and Pump-ID3)__ located in the directory. If you have time and are interested, you can look at:
* Merging the files - is the data continuous? are there duplicate timestamps in some of the files?
* Labeling the data for on, off, fan speed, volumetric flow rate, etc. 
* Plot the performance map. What do you notice if you filter out off modes or other irregular operating patterns?

In [ ]:
## Read in data - you can loop through the files in each directory, or read them in directly using the names
df_example = pd.read_csv("./Pump-ID1/01_20200223T113135_20200224T030130.csv")

## Some considerations:

Some of the sensors are inconsistent from the lab to real pump data. Keep this in mind when reusing code from previous section.

To estimate __mean indoor temperature__, take the mean of the sensors ["T_AO1", "T_AO2", "T_AO3"]

The __VFR coefficients__ differ by heat pump. To calculate VFR use the following coefficients and the functions in the next cell.
* For __Pump-ID1__: [-11409.0, 5241.7, -35.074]
* For __Pump-ID3__: [-121.145, 628.571, 394.170]

To run the functions above, apply using the indoor unit current (A_I), and enter the coefficients as a list: 

```compute_volumetric_flow_rate(df.A_I, [-11409.0, 5241.7, -35.074])```

In [ ]:
import pandas as pd
import plotly.express as px

import numpy as np
from typing import List
def convert_cfm_to_m3s(vfr_in_cfm: pd.Series) -> pd.Series:
    """
    Convert units [CFM] -> [m3s]
    (this should be done better in the furture - maybe pint)
    """
    return vfr_in_cfm * (1 / 60) * (1 / 35.3147)

def compute_volumetric_flow_rate(current: pd.Series, coeffs: List[float]) -> pd.Series:
    """
    Evaluate volumetric flow rate based on inside unit current
    according to a regression based on a polynomial fit.
    Args:
        current: indoor unit current
        coeffs: polynomial regression coefficients, see numpy.polyval for details
    Returns:
        vfr: pd.Series volumeflowrate in [m^3/s]
    """

    flow = pd.Series(np.polyval(coeffs, current.to_numpy()), index=current.index)
    return convert_cfm_to_m3s(flow).rename("VFR")